## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Petropavlovsk-Kamchatskiy,RU,53.0452,158.6483,41.00,75,68,2.24,broken clouds
1,1,Kirakira,SB,-10.4544,161.9205,83.98,71,65,4.81,broken clouds
2,2,Vao,NC,-22.6667,167.4833,75.67,76,84,13.73,broken clouds
3,3,Vydrino,RU,51.4850,104.8461,28.38,88,100,4.21,light snow
4,4,Kaitangata,NZ,-46.2817,169.8464,53.01,76,38,1.99,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Kirakira,SB,-10.4544,161.9205,83.98,71,65,4.81,broken clouds
2,2,Vao,NC,-22.6667,167.4833,75.67,76,84,13.73,broken clouds
6,6,Rikitea,PF,-23.1203,-134.9692,78.75,68,100,17.11,overcast clouds
16,16,Vaini,TO,-21.2000,-175.2000,82.40,83,40,3.44,scattered clouds
21,21,Saint-Philippe,RE,-21.3585,55.7679,77.00,83,75,16.11,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kirakira,SB,83.98,broken clouds,-10.4544,161.9205,
2,Vao,NC,75.67,broken clouds,-22.6667,167.4833,
6,Rikitea,PF,78.75,overcast clouds,-23.1203,-134.9692,
16,Vaini,TO,82.40,scattered clouds,-21.2000,-175.2000,
21,Saint-Philippe,RE,77.00,light rain,-21.3585,55.7679,
22,Avarua,CK,84.20,few clouds,-21.2078,-159.7750,
33,Mahebourg,MU,78.80,light rain,-20.4081,57.7000,
35,Kavieng,PG,80.15,light rain,-2.5744,150.7967,
37,Hilo,US,80.60,light rain,19.7297,-155.0900,
38,Butaritari,KI,82.15,light rain,3.0707,172.7902,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat}, {lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']

    # If an error is experienced, skip the city.
    except:
        print("Hotel not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
    
        

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
-----------------------------
Data Retrieval Complete      
-----------------------------


In [9]:
# 7. Drop the rows where there is no Hotel Name.
WeatherPy_vacation_df = hotel_df.dropna()

In [10]:
WeatherPy_vacation_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kirakira,SB,83.98,broken clouds,-10.4544,161.9205,SanBiz Lodge
2,Vao,NC,75.67,broken clouds,-22.6667,167.4833,Hotel Kou-Bugny
6,Rikitea,PF,78.75,overcast clouds,-23.1203,-134.9692,Pension Maro'i
16,Vaini,TO,82.40,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
21,Saint-Philippe,RE,77.00,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
22,Avarua,CK,84.20,few clouds,-21.2078,-159.7750,Paradise Inn
33,Mahebourg,MU,78.80,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
35,Kavieng,PG,80.15,light rain,-2.5744,150.7967,Nusa Island Retreat
37,Hilo,US,80.60,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
38,Butaritari,KI,82.15,light rain,3.0707,172.7902,Isles Sunset Lodge


In [11]:
# 8a. Create the output File (CSV)
output_data_file = '../Vacation_Search/WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
WeatherPy_vacation_df.to_csv(output_data_file, index_label='City_ID')

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} &deg;F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in WeatherPy_vacation_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = WeatherPy_vacation_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = WeatherPy_vacation_df[["Lat", "Lng"]]
max_temp = WeatherPy_vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))